## DAT405 Introduction to Data Science and AI
# Assignment 4: Spam classification using Naïve Bayes

Student name | Hours spent on the tasks
------------ | -------------
Lenia Malki | 10
Maële Belmont | 10

ask:
- binarize value

## Setup
Python modules need to be loaded to solve the tasks.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import os
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer

## Task 1 - Preprocessing: 
#### a. Note that the email files contain a lot of extra information, besides the actual message. Ignore that for now and run on the entire text. Further down (in the higher grade part), you will be asked to filter out the headers and footers. 

#### b. We don’t want to train and test on the same data. Split the spam and the ham datasets in a training set and a test set.

In [2]:
#Method which saves files of a directory to a dataframe
def getFiles(folderpath):
    filepaths  = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]
    df = pd.DataFrame(filepaths)
    return df

#Assign dataframes to variables
e_ham_files = getFiles('easy_ham')
h_ham_files = getFiles('hard_ham') 
spam_ham_files = getFiles('spam')

#Method which extract all messages from a dataframe containing file-paths
def getFileContent(dataFrame,k):
    messages = []
    for i in range(len(dataFrame)):
        filename = dataFrame.iloc[i,0]
        with open(filename, encoding="Latin-1") as f:
            messages.append([f.read(),k]) #The "k" indicates whether it is spam=1 or ham=0
    data = pd.DataFrame(messages, columns=["Content","Type"])
    return data

In [3]:
#Assigning file contents from directory to respective data frames
easyHamContent = getFileContent(e_ham_files,0)
hardHamContent = getFileContent(h_ham_files,0)
spamContent = getFileContent(spam_ham_files,1)

# Split data frames into training set and test set (70-30)
easyHamTrain, easyHamTest = train_test_split(easyHamContent, test_size=0.3, random_state=0)
hardHamTrain, hardHamTest = train_test_split(hardHamContent, test_size=0.3, random_state=0)
spamTrain, spamTest = train_test_split(spamContent, test_size=0.3, random_state=0)

## Task 2 - Write a Python program that:
#### a. Uses four datasets (hamtrain, spamtrain, hamtest, and spamtest)

#### b. Using a Naïve Bayes classifier (e.g. Sklearn), classifies the test sets and reports the  percentage of ham and spam test sets that were classified correctly. You can use CountVectorizer to transform the email texts into vectors. Please note that there are different types of Naïve Bayes Classifier in SKlearn (Document is available here). Test two of these classifiers: 1. Multinomial Naive Bayes and 2. Bernoulli Naive Bayes that are well suited for this problem. For the case of Bernoulli Naive Bayes you should use the parameter binarize to make the features binary. Discuss the differences between these two classifiers. 

In [20]:
def bayesClassifier(classiferType, hamTrain, spamTrain, hamTest, spamTest):
    
    #Setting up parameters for classifer
    x_train = pd.concat([hamTrain["Content"], spamTrain["Content"]])
    y_train = pd.concat([hamTrain["Type"], spamTrain["Type"]])
    x_test = pd.concat([hamTest["Content"], spamTest["Content"]])
    y_test = pd.concat([hamTest["Type"], spamTest["Type"]])
    
    #Convert a collection of text documents to a matrix of token counts.
    #Needed in order to convert string to float for fitting 
    vectorizer = CountVectorizer()
    vectorizer.fit(x_train)
    trainVector = vectorizer.transform(x_train)
    testVector = vectorizer.transform(x_test)
    
    if classiferType == "MNB":
        clf = MultinomialNB() # Multinomial Naive Bayes
    elif classiferType == "BNM":
        clf = BernoulliNB(fit_prior=False, binarize=1) # Bernouilli Naive Bayes + ADD parameter binarize to make the features binary

    #Fitting the model
    clf.fit(trainVector, y_train) 
    y_pred = clf.predict(testVector)

    # Model Accuracy, how often is the classifier correct?
    print("Accuracy with " +  str(classiferType) + " classifier: ", round(metrics.accuracy_score(y_test, y_pred)*100,2),'%\n')
    
    
    """
    Maybe add code for accuracy scores? Such as true positives, true negatives etc
    """

Differences between the two classifiers:







## Task 3 - Run your program on
#### i. Spam versus easy-ham

In [19]:
bayesClassifier("MNB", easyHamTrain, spamTrain, easyHamTest, spamTest)
bayesClassifier("BNM", easyHamTrain, spamTrain, easyHamTest, spamTest)

Accuracy with MNB classifier:  97.71 %

Accuracy with BNM classifier:  85.5 %



#### ii. Spam versus hard-ham

In [6]:
bayesClassifier("MNB", hardHamTrain, spamTrain, hardHamTest, spamTest)
bayesClassifier("BNM", hardHamTrain, spamTrain, hardHamTest, spamTest)

Accuracy with MNB classifier:  92.04 %

Accuracy with BNM classifier:  86.73 %



## Task 4 - To avoid classification based on common and uninformative words it is common to filter these out. 
#### a. Argue why this may be useful. Try finding the words that are too common/uncommon in the dataset. 

#### b. Use the parameters in Sklearn’s CountVectorizer to filter out these words. Run the updated program on your data and record how the results differ from 3. You have two options to do this in Sklearn: either using the words found in part (a) or letting Sklearn do it for you.

## Task 5 - Filter out the headers and the footers of the emails before you run on them. The format may vary somewhat between emails, which can make this a bit tricky, so perfect filtering is not required. Run your program again and answer the following questions: 
#### a. Does the result improve from 3 and 4?

#### b. The split of the data set into a training set and a test set can lead to very skewed results. Why is this, and do you have suggestions on remedies?

#### c. What do you expect would happen if your training set were mostly spam messages while your test set were mostly ham messages?